# Reconstruction Error vs all GT folds

## Imports, functions and constants

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init
#from torch.utils.data import *
from torch.utils.data import Dataset, DataLoader

from CNN import *
# from GradCAM import *
# from GradCAMUtils import *
from Utils import *

import numpy as np

import pandas as pd
import seaborn as sns

import os

from fastai import *
from fastai.text import *
from fastai.vision import *
from fastai.imports import *

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import umap

from matplotlib.widgets import Button
from matplotlib.widgets import TextBox
from matplotlib.ticker import MultipleLocator
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

from scipy.stats import norm
from scipy.stats import genextreme

In [5]:
"""Constants"""
# sequence length indicate the maximum length for all of the sequnence 626/798
SEQUENCE_LENGTH = 798

BATCH_SIZE = 256

vocab = {'C': [0,0,1], 'H': [0,1,0], 'E': [1,0,0], '-':[0,0,0]}

sns.set(rc={'figure.figsize':(15,15)})

model_path = Path("./Models/")
path = Path("./Datasets/")

#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

# Transform the labels from String to Integer via LabelEncoder
le_fold = preprocessing.LabelEncoder()
le_fam = preprocessing.LabelEncoder()

# torch.cuda.set_device()
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

cuda_gpu = False   #判断GPU是否存在可用

torch.cuda.empty_cache()

In [6]:
def reconstruction_error_calculation(model, df, le_fam, le_fold, cuda_gpu, criterion):
    gt_ds, gt_dl = Dataset_Loader(df, le_fam, le_fold, vocab, BATCH_SIZE=1, cuda = cuda_gpu)
    reconstruction_err = []
    for i, data in enumerate(gt_dl, 0):
        model.eval()
        xb, yb, p = data
        output = model(xb)
        xb = xb.float()
        loss = criterion(output, xb)/(p.sum())
        reconstruction_err.append([df.iloc[i].Name,df.iloc[i].fold,df.iloc[i].family, loss.item()])
    return pd.DataFrame(reconstruction_err, columns=["Name","Fold","Family","Err"])

def Plot_Len_Dis_Extreme(r_err, GT_val, interval1 = 0.95, interval2 = 0.99, bins_val=100):
    covMat = np.array(r_err["Err"], dtype=float)
    median = np.median(covMat)
    c, loc, scale = genextreme.fit(covMat, floc=median)

    min_extreme1,max_extreme1 = genextreme.interval(interval1,c,loc,scale)
    min_extreme2,max_extreme2 = genextreme.interval(interval2,c,loc,scale)
    
    x = np.linspace(min(covMat),max(covMat),2000)

    fig,ax = plt.subplots(1, 1)
    plt.xlim(0,0.4)
    plt.plot(x, genextreme.pdf(x, *genextreme.fit(covMat)))
    plt.hist(np.array(r_err["Err"], dtype=float),bins=100,alpha=0.7, density=True)
    plt.hist(np.asarray(GT_val["Err"]), edgecolor='k', alpha=0.35, bins=bins_val, density=True) 
    plt.xlabel('Lengths Counts')
    plt.ylabel('Probability')
    plt.title(r'max_extreme1=%.3f,max_extreme2=%.3f' %(max_extreme1, max_extreme2))
    plt.annotate('Max Extreme Value 1',xy=(max_extreme1,0),xytext=(max_extreme1,1),arrowprops=dict(arrowstyle="->",connectionstyle="arc3",color="black"),color="black")
    plt.annotate('Max Extreme Value 2',xy=(max_extreme2,0),xytext=(max_extreme2,1),arrowprops=dict(arrowstyle="->",connectionstyle="arc3",color="black"),color="black")
    plt.grid(True)
    median = GT_val.median()
    print("95% CI upper bound:",max_extreme1)
    print("99% CI upper bound:",max_extreme2)
    print("Median RE:",median.values)
    return max_extreme1, max_extreme2, median

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

In [7]:
pwd

'/Users/rtaujale/Dropbox (Edison_Lab@UGA)/Projects/GT_informatics/GT/GT_strML/Github_Folder/GT-CNN/Codes'

## Reading saved datasets and model

In [5]:
# Define criterion and read in the model
criterion = nn.MSELoss(reduction="sum")

f=open( "../PretrainedModels/Autoencoder_All.pickle", "rb" )
model = CPU_Unpickler(f).load()

In [6]:
# Reconstruction error for the training data (Saved in this file from model training)
df_rerr_all = pd.read_csv("../Data/rerr_gtAll_training.csv")

## For any new set of sequences

In [7]:
# Read in the test dataset
#df_test = pd.read_csv("../../")
# df_test = pd.read_csv("../output/gt105_preprocessed.csv")
# df_test = pd.read_csv("../output/gt44_preprocessed_domOnly.csv")
# df_test = pd.read_csv("../output/allgtu_domOnly.csv")
# df_test = pd.read_csv("../output/allgtu_newdomain_RegularCut.csv")
# df_test = pd.read_csv("../output/allgtu_newdomain_DomainOnlyCut.csv")
# df_test = pd.read_csv("../output/allgtu_DomainOnlyCut_processed.e2.csv")

# df_test = pd.read_csv("../Data/gtu_domainOnly/allgtu_DomainOnlyCut_processed.csv")
df_test = pd.read_csv("../Data/gtu_107_114/newgtus107-114_DomainOnlyCut_processed.csv")

df_test.shape

(679, 8)

In [8]:
df_test.columns

Index(['Name', 'fold', 'family', 'q3seq', 'rawseq', 'q3seqTokens',
       'rawseqTokens', 'paddings'],
      dtype='object')

In [9]:
# Calculate RE
rerr_test = reconstruction_error_calculation(model, df_test, le_fam, le_fold, cuda_gpu, criterion)

/Users/rtaujale/opt/anaconda3/envs/GTstrML/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [10]:
rerr_test.sort_values("Err")

,Name,Fold,Family,Err
198,GT111-u|BBA48052.1|-,u,GT111-u,0.060579
357,GT111-u|ADP12973.1|-,u,GT111-u,0.061297
667,GT111-u|ASQ29972.1|-,u,GT111-u,0.061854
182,GT113-u|AWN18940.1|-,u,GT113-u,0.061933
346,GT113-u|BAQ95843.1|-,u,GT113-u,0.063663
471,GT111-u|CAO96386.1|-,u,GT111-u,0.063695
156,GT111-u|ATZ11949.1|-,u,GT111-u,0.064268
300,GT111-u|AVG77533.1|-,u,GT111-u,0.067066
657,GT111-u|AXR42330.1|-,u,GT111-u,0.068397
39,GT107-B|APW67013|A.spLPB0137_,u,GT107-u,0.068918


In [11]:
df=rerr_test.groupby('Family').median().reset_index().sort_values('Err')
df

,Family,Err
3,GT113-u,0.098003
2,GT112-u,0.100261
0,GT107-u,0.102565
4,GT114-u,0.108057
1,GT111-u,0.116901


In [12]:
# Save RE for the test dataset to file
# rerr_test.to_csv("../Data/gtu_domainOnly/results_allgtu_DomainOnlyCut_allRE.csv",index=False)
rerr_test.to_csv("../Data/gtu_107_114/results_newgtu_DomainOnlyCut_allRE.csv",index=False)


In [13]:
# df.to_csv("../Data/gtu_domainOnly/results_allgtu_DomainOnlyCut_medianRE.csv",index=False)
df.to_csv("../Data/gtu_107_114/results_newgtu_DomainOnlyCut_medianRE.csv",index=False)

## Additional analysis of results

Plotting the results:<br>
Can be done in a separate notebook "RE_analysis.ipynb" - (section Main Folds) <br>
Done this way to isolate parts of the workflow that requre a gpu to this notebook.

In [29]:
rerr_gtu=pd.read_csv("../../results/reconstruction/rerr_data/rerr_gtAll_test_new.csv")

In [82]:
## Caluclate median for each group (fold/family)
df=rerr_test.groupby('Family').median().reset_index().sort_values('Err')

## Add sequence counts for each family
df1 = pd.DataFrame(columns=['Family','Counts'])
fam=rerr_test['Family'].value_counts().index.tolist()
ct=rerr_test['Family'].value_counts()
for i,f in enumerate(fam):
    df1=df1.append(pd.DataFrame([[f,ct[i]]],columns=['Family','Counts']))
df=df.join(df1.set_index('Family'), on='Family')

# df.sort_values("Counts")
df

,Family,Err,Counts
9,GT110-u,0.058360,172
4,GT105-u,0.060304,148
15,GT53-u,0.067780,247
21,GT89-u,0.068232,232
20,GT76-u,0.070019,164
7,GT109-u,0.074860,33
3,GT103-u,0.075328,20
13,GT44-u,0.079880,93
27,GT98-u,0.080681,35
1,GT101-u,0.082473,106


In [83]:
df.to_csv("../output/results_allgtu_newdomain_DomainOnlyCut_medianREWithCounts.csv")

In [106]:
# How many for one family
rerr_gtu[rerr_gtu["Name"].str.contains("GT44-u")].shape

NameError: name 'rerr_gtu' is not defined

In [84]:
## Check rerr for any single family
# rerr_test[rerr_test['Family'].str.contains('GT71-u')].sort_values('Err')
rerr_test[rerr_test['Name'].str.contains('CAP80794')].sort_values('Err')

,Name,Fold,Family,Err
1059,GT71-u|CAP80794|P.rubensWisconsin54-1255_Fungi,u,GT71-u,0.12515


In [84]:
# Get sequence lengths with RE
r1=rerr_test.join(df_test.set_index('Name'), on='Name')
r1['len']=r1.apply(lambda x: len(x['rawseq'].replace("-", "")), axis=1)
r2=r1[['Name','Family','Err','len']]
r2[r2['Family'].str.contains('GT48-u')].sort_values('Err')

,Name,Family,Err,len
2255,GT48-u|ACY01929|B.vulgaris_Streptophyta,GT48-u,0.061599,609
2295,GT48-u|ACS36249|H.vulgare_Streptophyta,GT48-u,0.062413,425
3142,GT48-u|BAX37083|E.gracilisZ_Euglenozoa,GT48-u,0.065087,676
1687,GT48-u|CCF60041|K.africanaCBS2517_Fungi,GT48-u,0.066549,586
616,GT48-u|SGZ48571|C.intermediaPYCC4715_Fungi,GT48-u,0.071439,701
3369,GT48-u|BAX37082|E.gracilisZ_Euglenozoa,GT48-u,0.072406,419
559,GT48-u|AAD31571|A.thaliana_Streptophyta,GT48-u,0.072974,512
1936,GT48-u|CDR42478|C.fabianiiYJS4271_Fungi,GT48-u,0.075834,588
2767,GT48-u|AHA86554|A.protothecoides0710_Chlorophyta,GT48-u,0.076031,696
230,GT48-u|ACF22801|C.tropicalisATCC750_Fungi,GT48-u,0.076277,703


In [73]:
r2.shape

(3860, 4)

In [75]:
df_test.shape

(3858, 8)

In [85]:
r2.to_csv("../output/results_allgtu_newdomain_DomainOnlyCut_withLen.csv",index=False)